In [ ]:
import langchain
import os
os.environ['GOOGLE_API_KEY']="GOOGLE_API_KEY"
os.environ['LANGCHAIN_API_KEY']='LANGCHAIN_API_KEY'
os.environ['LANGCHAIN_TRACKING_V2']='TRUE'
os.environ['LANGCHAIN_PROJECT']='langchain-course'

Call LLM

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI,GoogleGenerativeAIEmbeddings
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
llm_response=llm.invoke('Tell me a joke on Donalt Trump')
llm_response

AIMessage(content='Why did Donald Trump bring a ladder to the press conference?  \n\nBecause he heard the polls were dropping!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run-3a2e75e1-9e76-4a60-8350-f66bf0405130-0', usage_metadata={'input_tokens': 8, 'output_tokens': 23, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}})

Parsing Output - Used to structue the output of Language Models

In [5]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
output_parser.invoke(llm_response)

'Why did Donald Trump bring a ladder to the press conference?  \n\nBecause he heard the polls were dropping!'

Simple Chain - Sequence of operations that process input and structure output.

In [6]:
chains= llm | output_parser
chains.invoke('Tell me a joke on Virat Kohli')

'Why was Virat Kohli sad after the match?  Because he only scored 99 runs... he was one short of a century!'

Structured Output- It ensures that generated output follow the specific format.

In [7]:
from typing import List
from pydantic import BaseModel, Field

class mobilereview(BaseModel):
    phone_model:str=Field(description='Name and Model of the Phone')
    rating:float=Field(description='Overall rating out of 5')
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm=llm.with_structured_output(mobilereview)
output=structured_llm.invoke(review_text)
output


mobilereview(phone_model='"Galaxy S21"', rating=4.0, pros=['"Gorgeous screen", "Amazing camera, especially at night", "Solid battery life"'], cons=['"Pricey", "No charger included", "New button layout"'], summary='"Great phone with a few minor drawbacks."')

In [8]:
output.pros

['"Gorgeous screen", "Amazing camera, especially at night", "Solid battery life"']

Prompt Template - Helps structure prompts dynamically by injecting user inputs into predefined templates.

In [9]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template('Tell me a short joke about {topic}')
prompt.invoke({'topic':'programming'})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about programming', additional_kwargs={}, response_metadata={})])

In [10]:
chain =prompt | llm | output_parser
chain.invoke({'topic':'Cricket'})

'Why did the cricket get a bad grade in school?\n\nBecause he was always stumped!'

In [11]:
#!pip install langchain-google-genai

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

prompt=ChatPromptTemplate.from_template('Tell me a joke about{topic}')

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

output_parser=StrOutputParser()

chain= prompt | llm | output_parser

chain.invoke({'topic':'Engineers'})

"Why was the engineer sad?  Because they didn't get the *array* they wanted."

LLM Message

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage,SystemMessage

system_message=SystemMessage(content='You are helpful Assistent that tells joke')
human_message=HumanMessage(content='Tell me about Programming')
llm.invoke([system_message,human_message])

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run-b0e018db-4bfc-4175-8a1d-d96f2ab22a23-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

In [14]:
template=ChatPromptTemplate([
    ('system','You are helpful assistent that tells joke'),
    ('human','Tell me about {topic}')
])

prompt_value=template.invoke(
    {
        'topic':'Programming'
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are helpful assistent that tells joke', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about Programming', additional_kwargs={}, response_metadata={})])

In [15]:
llm.invoke(prompt_value)

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run-7878a280-7279-48be-a11d-eb740c9bbc33-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

# RAG

In [20]:
# Will Load the data and Conver it into chunk size

from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI ,GoogleGenerativeAIEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

pdf_loader=TextLoader(r'/content/india.txt')
pdf=pdf_loader.load()

print(len(pdf))

splits=text_splitter.split_documents(pdf)
print(f"Split the pdf into {len(splits)} Chunks")


1
Split the pdf into 270 Chunks


In [21]:
#! pip install langchain_community

In [22]:
splits[55].metadata

{'source': '/content/india.txt'}

In [ ]:
# Convert the chunks into Embeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
pdf_embedding=embedding_function.embed_documents([split.page_content for split in splits])

In [24]:
pdf_embedding[0]

[-0.002303160261362791,
 0.06091883033514023,
 -0.015139984898269176,
 0.05076989158987999,
 -0.05676563084125519,
 -0.04324696213006973,
 -0.07008883357048035,
 -0.001948286546394229,
 -0.10021170228719711,
 0.027651198208332062,
 0.06501525640487671,
 -0.12778808176517487,
 0.06735087931156158,
 0.021507401019334793,
 0.020255064591765404,
 -0.03215751424431801,
 -0.06417519599199295,
 -0.042843468487262726,
 0.055180616676807404,
 -0.06490717083215714,
 -0.022579826414585114,
 -0.0024166309740394354,
 -0.0021075240802019835,
 -0.05484858900308609,
 0.04776130989193916,
 -0.013173849321901798,
 0.07596557587385178,
 -0.05858813598752022,
 0.020324815064668655,
 0.07963310182094574,
 0.016963548958301544,
 0.03515448048710823,
 0.06512057781219482,
 0.008619350381195545,
 -0.023185336962342262,
 0.06382950395345688,
 -0.006765254773199558,
 0.0495096892118454,
 -0.0026656980626285076,
 -0.05634189024567604,
 0.0032924236729741096,
 -0.006354631390422583,
 0.052617765963077545,
 -0.036

In [27]:
# Will store this embeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
embedding_function=SentenceTransformerEmbeddings()
vectorestore=Chroma.from_documents(documents=splits,embedding=embedding_function,persist_directory=r"./chroma_db")

<ipython-input-27-998a204828be>:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_function=SentenceTransformerEmbeddings()


In [28]:
#! pip install chromadb

In [31]:
retriever = vectorestore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("Tell me about Kashmir?")


[Document(metadata={'source': '/content/india.txt'}, page_content='ISBN\n978-8131728888\n.\n338.\nMadhya Pradesh National Means-Cum-Merit Scholarship Exam (Warren Hasting\'s system ofDual Government) (https://books.google.com/books?id=QnyaLNskRfEC&pg=PA11)\n. UpkarPrakashan. 2009. pp. 11–.\nISBN\n978-81-7482-744-9\n.\n339.\nBlack, Jeremy (2006),\nA Military History of Britain: from 1775 to the Present (https://books.goog\nle.com/books?id=hNVtQY4sXYMC&q=9780275990398)\n, Westport, Conn.: GreenwoodPublishing Group, p. 78,\nISBN\n978-0-275-99039-8\n340.\n"Treaty of Amritsar" (https://web.archive.org/web/20140826235201/http://www.kashmir-issue.co\nm/images3/treatyOfamritsar.pdf)\n(PDF)\n. Archived from\nthe original (http://www.kashmir-issue.co\nm/images3/treatyOfamritsar.pdf)\n(PDF)\non 26 August 2014\n. Retrieved\n25 August\n2014\n.\n341.\nRai, Mridu (2004).\nHindu Rulers, Muslim Subjects: Islam, Rights, and the History of Kashmir (ht\ntps://books.google.com/books?id=x5azvT2hjW0C)\n. Pri

In [32]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [33]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("Tell me about Kashmir?")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\n[Document(metadata={\'source\': \'/content/india.txt\'}, page_content="(1772–1818)left it in control of large areas of India south of the\\nSutlejRiver\\n. With the defeat of the\\nMarathas\\n, no native powerrepresented a threat for the company any longer.\\n[\\n350\\n]\\nThe expansion of the company\'s power chiefly took twoforms. The first of these was the outright annexation ofIndian states and subsequent direct governance of theunderlying regions that collectively came to comprise BritishIndia. The annexed regions included the\\nNorth-WesternProvinces\\n(comprising\\nRohilkhand\\n,\\nGorakhpur\\n, and the\\nDoab\\n) (1801), Delhi (1803), Assam (\\nAhom Kingdom\\n1828)and\\nSindh\\n(1843).\\nPunjab\\n,\\nNorth-West Frontier Province\\n, and\\nKashmir\\nwere annexed after the\\nAnglo-Sikh Wars\\nin 1849–56(Period of tenure of Marquess of Dalhousie GovernorGeneral). However, Kash

In [34]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [35]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("Tell me about Kashmir?")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\nThe second form of asserting power involved treaties in which Indian rulers acknowledged thecompany\'s\nhegemony\nin return for limited internal\nautonomy\n. Since the company operated underfinancial constraints, it had to set up\npolitical\nunderpinnings for its rule.\n[\n351\n]\nThe most importantsuch support came from the\nsubsidiary alliances\nwith Indian princes.\n[\n351\n]\nIn the early 19thcentury, the territories of these princes accounted for two-thirds of India.\n[\n351\n]\nWhen an Indianruler who was able to secure his territory wanted to enter such an alliance, the company welcomedit as an economical method of indirect rule that did not involve the economic costs of directadministration or the political costs of gaining the support of alien subjects.\n[\n352\n]\nIn return, the company undertook the "defense of these subordinate allies and treated them withtraditional re

In [37]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "Tell me about Kashmir?"
response = rag_chain.invoke(question)
print(response)

Based on the provided text, Kashmir is discussed in two sources:

* **Rai, Mridu (2004). *Hindu Rulers, Muslim Subjects: Islam, Rights, and the History of Kashmir*. Princeton University Press. pp. 27, 133. ISBN 978-0-691-11688-4.**  This book examines Islam, rights, and the history of Kashmir.

* A PDF titled "Treaty of Amritsar" (archived) is mentioned, which may contain information about Kashmir, although the content of the PDF cannot be assessed from the given context.


Conversational RAG

In [38]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [39]:
chat_history

[HumanMessage(content='Tell me about Kashmir?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Based on the provided text, Kashmir is discussed in two sources:\n\n* **Rai, Mridu (2004). *Hindu Rulers, Muslim Subjects: Islam, Rights, and the History of Kashmir*. Princeton University Press. pp. 27, 133. ISBN 978-0-691-11688-4.**  This book examines Islam, rights, and the history of Kashmir.\n\n* A PDF titled "Treaty of Amritsar" (archived) is mentioned, which may contain information about Kashmir, although the content of the PDF cannot be assessed from the given context.', additional_kwargs={}, response_metadata={})]

In [44]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt=ChatPromptTemplate.from_messages([
    ('system',contextualize_q_system_prompt),
    MessagesPlaceholder('chat_history'),
    ('human','{input}'),
])

contextualize_chain=contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({'input':'What is capital ?','chat_history':chat_history})

'What is capital?'

In [45]:
from langchain.chains import create_history_aware_retriever
history_aware_retriver=create_history_aware_retriever(
    llm,retriever,contextualize_q_prompt
)
history_aware_retriver.invoke({'input':'What is capital','chat_history':chat_history})

[Document(metadata={'source': '/content/india.txt'}, page_content='ISBN\n978-1-4051-9509-6\n126.\nA Social History of Early India\nby Brajadulal Chattopadhyaya p. 259\n127.\n"The World Economy (GDP) : Historical Statistics by Professor Angus Maddison" (http://www.th\neworldeconomy.org/MaddisonTables/MaddisontableB-18.pdf)\n(PDF)\n. World Economy. 17September 2006\n. Retrieved\n21 May\n2013\n.\n128.\nMaddison, Angus (2006).\nThe World Economy – Volume 1: A Millennial Perspective andVolume 2: Historical Statistics (http://www.oecdbookshop.org/oecd/display.asp?K=5L9ZBQKL5\nRLW&lang=EN&sort=sort_date%2Fd&stem=true&sf1=Title&st1=world+economy&sf3=Subject\nCode&sp1=not&st4=E4+or+E5+or+P5&sf4=SubVersionCode&ds=world+economy%3B+All+Su\nbjects%3B+&m=3&dc=26&plang=en)\n. OECD Publishing by\nOrganisation for Economic Co-operation and Development\n. p. 656.\nISBN\n978-92-64-02262-1\n.\n129.\nStadtner, Donald (1975). "A Śuṅga Capital from Vidiśā".\nArtibus Asiae\n.\n37\n(1/2):\n101–\n104.\ndoi\n:\n

In [46]:
retriever.invoke('What is capital ?')

[Document(metadata={'source': '/content/india.txt'}, page_content='ISBN\n978-1-4051-9509-6\n126.\nA Social History of Early India\nby Brajadulal Chattopadhyaya p. 259\n127.\n"The World Economy (GDP) : Historical Statistics by Professor Angus Maddison" (http://www.th\neworldeconomy.org/MaddisonTables/MaddisontableB-18.pdf)\n(PDF)\n. World Economy. 17September 2006\n. Retrieved\n21 May\n2013\n.\n128.\nMaddison, Angus (2006).\nThe World Economy – Volume 1: A Millennial Perspective andVolume 2: Historical Statistics (http://www.oecdbookshop.org/oecd/display.asp?K=5L9ZBQKL5\nRLW&lang=EN&sort=sort_date%2Fd&stem=true&sf1=Title&st1=world+economy&sf3=Subject\nCode&sp1=not&st4=E4+or+E5+or+P5&sf4=SubVersionCode&ds=world+economy%3B+All+Su\nbjects%3B+&m=3&dc=26&plang=en)\n. OECD Publishing by\nOrganisation for Economic Co-operation and Development\n. p. 656.\nISBN\n978-92-64-02262-1\n.\n129.\nStadtner, Donald (1975). "A Śuṅga Capital from Vidiśā".\nArtibus Asiae\n.\n37\n(1/2):\n101–\n104.\ndoi\n:\n